In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Read the dataset
dataset_train = pd.read_csv("/content/heart.csv")

# Extract features and labels
X = dataset_train.drop('HeartDisease', axis=1)
y = dataset_train['HeartDisease']

# Encode categorical variables (if any)
X = pd.get_dummies(X, columns=['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data into sequences suitable for RNN
sequence_length = 10

def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:i+sequence_length])
        labels.append(data[i+sequence_length-1, 0])  # Adjusted indexing for labels
    return np.array(sequences), np.array(labels)

X_train_seq, y_train_seq = create_sequences(X_train_scaled, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, sequence_length)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(128, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
y_pred_probs = model.predict(X_test_seq)
y_pred = (y_pred_probs > 0.5).astype(int)

# Ensure y_test_seq is binary
y_test_binary = (y_test_seq > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test_binary, y_pred)
precision = precision_score(y_test_binary, y_pred)
recall = recall_score(y_test_binary, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')


Epoch 1/50
37/37 [==============================] - 5s 28ms/step - loss: 0.2585 - accuracy: 0.0000e+00 - val_loss: -0.5051 - val_accuracy: 0.0000e+00
Epoch 2/50
37/37 [==============================] - 1s 19ms/step - loss: -1.8065 - accuracy: 0.0000e+00 - val_loss: -3.7606 - val_accuracy: 0.0000e+00
Epoch 3/50
37/37 [==============================] - 1s 22ms/step - loss: -6.3582 - accuracy: 0.0000e+00 - val_loss: -9.5794 - val_accuracy: 0.0000e+00
Epoch 4/50
37/37 [==============================] - 1s 36ms/step - loss: -14.9072 - accuracy: 0.0000e+00 - val_loss: -19.6069 - val_accuracy: 0.0000e+00
Epoch 5/50
37/37 [==============================] - 1s 23ms/step - loss: -27.0850 - accuracy: 0.0000e+00 - val_loss: -33.3901 - val_accuracy: 0.0000e+00
Epoch 6/50
37/37 [==============================] - 1s 18ms/step - loss: -42.7893 - accuracy: 0.0000e+00 - val_loss: -48.7792 - val_accuracy: 0.0000e+00
Epoch 7/50
37/37 [==============================] - 1s 20ms/step - loss: -60.2521 - accur

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Read the dataset
dataset_train = pd.read_csv("/content/heart.csv")

# Data preprocessing
X = dataset_train.drop('HeartDisease', axis=1)
y = dataset_train['HeartDisease']
X = pd.get_dummies(X, columns=['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], drop_first=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape input data for RNN
X_train_reshaped = X_train.reshape(-1, X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(-1, X_test.shape[1], 1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(100, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
y_pred_probs = model.predict(X_test_reshaped)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

from sklearn.metrics import precision_score, recall_score

# Calculate precision
precision = precision_score(y_test, y_pred)

# Calculate recall
recall = recall_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)


Epoch 1/100
37/37 [==============================] - 2s 28ms/step - loss: 0.4082 - accuracy: 0.8245 - val_loss: 0.4026 - val_accuracy: 0.8435
Epoch 2/100
37/37 [==============================] - 1s 15ms/step - loss: 0.3289 - accuracy: 0.8603 - val_loss: 0.4102 - val_accuracy: 0.8435
Epoch 3/100
37/37 [==============================] - 0s 12ms/step - loss: 0.3337 - accuracy: 0.8688 - val_loss: 0.3934 - val_accuracy: 0.8571
Epoch 4/100
37/37 [==============================] - 0s 13ms/step - loss: 0.3369 - accuracy: 0.8603 - val_loss: 0.4171 - val_accuracy: 0.8435
Epoch 5/100
37/37 [==============================] - 0s 12ms/step - loss: 0.3171 - accuracy: 0.8620 - val_loss: 0.4278 - val_accuracy: 0.8299
Epoch 6/100
37/37 [==============================] - 0s 12ms/step - loss: 0.3300 - accuracy: 0.8739 - val_loss: 0.3781 - val_accuracy: 0.8639
Epoch 7/100
37/37 [==============================] - 0s 13ms/step - loss: 0.3145 - accuracy: 0.8671 - val_loss: 0.3964 - val_accuracy: 0.8571
Epoch 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Read the dataset
dataset_train = pd.read_csv("/content/heart.csv")

# Encode categorical variables
label_encoder = LabelEncoder()
dataset_train['Sex'] = label_encoder.fit_transform(dataset_train['Sex'])
dataset_train['ChestPainType'] = label_encoder.fit_transform(dataset_train['ChestPainType'])
dataset_train['FastingBS'] = label_encoder.fit_transform(dataset_train['FastingBS'])
dataset_train['RestingECG'] = label_encoder.fit_transform(dataset_train['RestingECG'])
dataset_train['ExerciseAngina'] = label_encoder.fit_transform(dataset_train['ExerciseAngina'])
dataset_train['ST_Slope'] = label_encoder.fit_transform(dataset_train['ST_Slope'])

# Extract features and labels
X = dataset_train.drop('HeartDisease', axis=1)
y = dataset_train['HeartDisease']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape input data for RNN
X_train_reshaped = X_train_scaled.reshape(-1, X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(-1, X_test_scaled.shape[1], 1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(100, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
y_pred_probs = model.predict(X_test_reshaped)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Epoch 1/100
37/37 [==============================] - 4s 26ms/step - loss: 0.4373 - accuracy: 0.8092 - val_loss: 0.4591 - val_accuracy: 0.7755
Epoch 2/100
37/37 [==============================] - 1s 15ms/step - loss: 0.3864 - accuracy: 0.8211 - val_loss: 0.4362 - val_accuracy: 0.8231
Epoch 3/100
37/37 [==============================] - 1s 14ms/step - loss: 0.3637 - accuracy: 0.8518 - val_loss: 0.4008 - val_accuracy: 0.8367
Epoch 4/100
37/37 [==============================] - 1s 30ms/step - loss: 0.3552 - accuracy: 0.8552 - val_loss: 0.4342 - val_accuracy: 0.8095
Epoch 5/100
37/37 [==============================] - 1s 19ms/step - loss: 0.3519 - accuracy: 0.8569 - val_loss: 0.4110 - val_accuracy: 0.8435
Epoch 6/100
37/37 [==============================] - 1s 18ms/step - loss: 0.3590 - accuracy: 0.8313 - val_loss: 0.4168 - val_accuracy: 0.8435
Epoch 7/100
37/37 [==============================] - 1s 15ms/step - loss: 0.3516 - accuracy: 0.8467 - val_loss: 0.4217 - val_accuracy: 0.8299
Epoch 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Read the dataset
dataset_train = pd.read_csv("/content/heart.csv")

# Extract features and labels
X = dataset_train.drop('HeartDisease', axis=1)
y = dataset_train['HeartDisease']

# Encode categorical variables (if any)
X = pd.get_dummies(X, columns=['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape input data for RNN
X_train_reshaped = X_train_scaled.reshape(-1, X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(-1, X_test_scaled.shape[1], 1)

# Build the RNN model
model = Sequential()
model.add(SimpleRNN(200, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
y_pred_probs = model.predict(X_test_reshaped)
y_pred = (y_pred_probs > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)



# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)


print(f'Accuracy: {accuracy}')


Epoch 1/100
19/19 [==============================] - 6s 74ms/step - loss: 0.5126 - accuracy: 0.7462 - val_loss: 0.4007 - val_accuracy: 0.8435
Epoch 2/100
19/19 [==============================] - 1s 29ms/step - loss: 0.3486 - accuracy: 0.8569 - val_loss: 0.3877 - val_accuracy: 0.8571
Epoch 3/100
19/19 [==============================] - 0s 26ms/step - loss: 0.3308 - accuracy: 0.8603 - val_loss: 0.3947 - val_accuracy: 0.8503
Epoch 4/100
19/19 [==============================] - 1s 27ms/step - loss: 0.3289 - accuracy: 0.8603 - val_loss: 0.3883 - val_accuracy: 0.8639
Epoch 5/100
19/19 [==============================] - 0s 23ms/step - loss: 0.3101 - accuracy: 0.8705 - val_loss: 0.3877 - val_accuracy: 0.8707
Epoch 6/100
19/19 [==============================] - 0s 26ms/step - loss: 0.3099 - accuracy: 0.8705 - val_loss: 0.3919 - val_accuracy: 0.8639
Epoch 7/100
19/19 [==============================] - 0s 24ms/step - loss: 0.3090 - accuracy: 0.8739 - val_loss: 0.3870 - val_accuracy: 0.8639
Epoch 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the ANN model architecture
model = Sequential([
    Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Evaluate the model
y_pred_probs = model.predict(X_test_scaled)
y_pred = (y_pred_probs > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Epoch 1/100
19/19 [==============================] - 2s 23ms/step - loss: 0.5709 - accuracy: 0.7172 - val_loss: 0.4624 - val_accuracy: 0.8367
Epoch 2/100
19/19 [==============================] - 0s 9ms/step - loss: 0.3571 - accuracy: 0.8603 - val_loss: 0.4023 - val_accuracy: 0.8435
Epoch 3/100
19/19 [==============================] - 0s 8ms/step - loss: 0.3111 - accuracy: 0.8807 - val_loss: 0.3928 - val_accuracy: 0.8571
Epoch 4/100
19/19 [==============================] - 0s 8ms/step - loss: 0.2856 - accuracy: 0.8910 - val_loss: 0.3967 - val_accuracy: 0.8503
Epoch 5/100
19/19 [==============================] - 0s 8ms/step - loss: 0.2733 - accuracy: 0.8944 - val_loss: 0.3963 - val_accuracy: 0.8367
Epoch 6/100
19/19 [==============================] - 0s 14ms/step - loss: 0.2576 - accuracy: 0.9012 - val_loss: 0.3888 - val_accuracy: 0.8435
Epoch 7/100
19/19 [==============================] - 0s 6ms/step - loss: 0.2467 - accuracy: 0.9046 - val_loss: 0.3929 - val_accuracy: 0.8367
Epoch 8/100